# Buffer Overflow Protections
- modern kernel and compilers provide various proctections from bufferoverlow attacks
- Reference: https://ocw.cs.pub.ro/courses/cns/labs/lab-06

## 1. ASLR - Address Space Layout Randomization
- Kernel specific protection
- loads the stack, data and program in random memory locations making it difficult to guess their addresses

- the following segments are possible to randomize on x86:
    - **stack** is easily randomizable, as all stack addresses are relative to `esp` or `ebp`
    - **Data segment** may be randomized, if e.g., the data segment is set to a random value
    - **Code** can only be randomized by comiling the program as Position Independent Code/Position Independent Executable
       - default for shared libraries, but otherwise executable code is usually placed at fixed addresses

### ASLR values
- Linux allows 3 options:
    - 0: disabled
    - 1: randomize stack, vdso, libraries (except data? not sure...)
    - 2: enabled (including data segment - default value)
    
### checking ASLR
- compile and run demos/aslr.cpp program to check ASLR
- run gdb-peda 

    ```bash
    aslr # run aslr command
    ASLR is OFF # output
    ```

In [1]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 0


In [2]:
! cat demos/aslr/aslr.cpp

// aslr.cpp - program to check address space layout randomization
// compile and run the program multiple times
// if you see same value for $esp, ASLR is disabled
// $ g++ -g -m 32 -o aslr.exe aslr.cpp
#include <stdio.h>
#include <iostream>
using namespace std;

int global_var = 100; // data segment
char global_name[20]; // bss segment

int main(void) {
    char buffer[] = "Hello World!";
    register int esp asm("esp");
    register int ebp asm("ebp");
    cout << "Registers info:\n";
    cout << "===============\n";
    printf("$esp = %p\n", esp);
    printf("$ebp = %p\n", ebp);
    cout << "\nData and BSS segement\n";
    cout << "======================\n";
    printf("global_var is at: %p\n", &global_var);
    printf("global_name is at: %p\n", global_name);
    cout << "\nStack variable:\n";
    cout << "===============\n";
    printf("buffer is at: %p\n", buffer);
    cout << "\nCode segment:\n";
    cout << "=============\n";
    printf("main is at %p\n", main);
    cout << "\nS

In [3]:
%%bash
input="demos/aslr/aslr.cpp"
output="aslr.exe"
g++ -m32 $input -o $output

In [4]:
! ./aslr.exe | tee aslr2-1.txt

Registers info:
$esp = 0xffffc350
$ebp = 0xffffc368

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc353

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c50f80
cout is at 0xf7faec00


In [5]:
! ./aslr.exe | tee aslr2-2.txt

Registers info:
$esp = 0xffffc350
$ebp = 0xffffc368

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc353

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c50f80
cout is at 0xf7faec00


In [6]:
# run the diff of aslr2-1.txt and aslr2-2.txt
! diff aslr2-1.txt aslr2-2.txt

In [7]:
# ldd invokes the standard dynamic linker with LD_TRACE_LOADED OBJECTS
# causes the dynamic linker to inspect the program's dynamic dependencies and the objects
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fd1000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7de1000)
	libc.so.6 => /lib32/libc.so.6 (0xf7bf8000)
	libm.so.6 => /lib32/libm.so.6 (0xf7af4000)
	/lib/ld-linux.so.2 (0xf7fd3000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7ad5000)


In [8]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fd1000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7de1000)
	libc.so.6 => /lib32/libc.so.6 (0xf7bf8000)
	libm.so.6 => /lib32/libm.so.6 (0xf7af4000)
	/lib/ld-linux.so.2 (0xf7fd3000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7ad5000)


### Set ASLR to option 1
- randomize stack, vdso, libraries (except data?)
- actually, global data and bss segments are also randomized!

In [12]:
! echo kali | sudo -S bash -c 'echo 1 | sudo -S tee /proc/sys/kernel/randomize_va_space'

[sudo] password for kali: 1


In [13]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 1


In [14]:
! ./aslr.exe | tee aslr1-1.txt

Registers info:
$esp = 0xffa8b720
$ebp = 0xffa8b738

Data and BSS segement
global_var is at: 0x565be024
global_name is at: 0x565be02c

Stack variable:
buffer is at: 0xffa8b723

Code segment:
main is at 0x565bb1b9

Shared library code
printf is at 0xf7c4ff80
cout is at 0xf7fadc00


In [15]:
! ./aslr.exe | tee aslr1-2.txt

Registers info:
$esp = 0xffd808d0
$ebp = 0xffd808e8

Data and BSS segement
global_var is at: 0x565a7024
global_name is at: 0x565a702c

Stack variable:
buffer is at: 0xffd808d3

Code segment:
main is at 0x565a41b9

Shared library code
printf is at 0xf7ba2f80
cout is at 0xf7f00c00


In [16]:
! diff aslr1-1.txt aslr1-2.txt

3,4c3,4
< $esp = 0xffa8b720
< $ebp = 0xffa8b738
---
> $esp = 0xffd808d0
> $ebp = 0xffd808e8
8,9c8,9
< global_var is at: 0x565be024
< global_name is at: 0x565be02c
---
> global_var is at: 0x565a7024
> global_name is at: 0x565a702c
13c13
< buffer is at: 0xffa8b723
---
> buffer is at: 0xffd808d3
17c17
< main is at 0x565bb1b9
---
> main is at 0x565a41b9
21,22c21,22
< printf is at 0xf7c4ff80
< cout is at 0xf7fadc00
---
> printf is at 0xf7ba2f80
> cout is at 0xf7f00c00


In [17]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7ed7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7ce7000)
	libc.so.6 => /lib32/libc.so.6 (0xf7afe000)
	libm.so.6 => /lib32/libm.so.6 (0xf79fa000)
	/lib/ld-linux.so.2 (0xf7ed9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf79db000)


In [18]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7f64000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7d74000)
	libc.so.6 => /lib32/libc.so.6 (0xf7b8b000)
	libm.so.6 => /lib32/libm.so.6 (0xf7a87000)
	/lib/ld-linux.so.2 (0xf7f66000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7a68000)


### Set ASLR to option 0
- disable ASLR

In [19]:
! echo kali | sudo -S bash -c 'echo 0 | sudo tee /proc/sys/kernel/randomize_va_space'

[sudo] password for kali: 0


In [20]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 0


In [21]:
! ./aslr.exe | tee aslr0-1.txt

Registers info:
$esp = 0xffffc370
$ebp = 0xffffc388

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc373

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c50f80
cout is at 0xf7faec00


In [22]:
! ./aslr.exe | tee aslr0-2.txt

Registers info:
$esp = 0xffffc370
$ebp = 0xffffc388

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc373

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c50f80
cout is at 0xf7faec00


In [23]:
! diff aslr0-1.txt aslr0-2.txt
# no difference!

In [25]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fd1000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7de1000)
	libc.so.6 => /lib32/libc.so.6 (0xf7bf8000)
	libm.so.6 => /lib32/libm.so.6 (0xf7af4000)
	/lib/ld-linux.so.2 (0xf7fd3000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7ad5000)


In [26]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fd1000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7de1000)
	libc.so.6 => /lib32/libc.so.6 (0xf7bf8000)
	libm.so.6 => /lib32/libm.so.6 (0xf7af4000)
	/lib/ld-linux.so.2 (0xf7fd3000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7ad5000)


#### Allow ptrace processes
**ptrace** - allows parent process to observe and control the execution of another process
- used primarily to implement breakpoint debugging and system call tracing

In [7]:
# check the ptrace value
! echo kali | sudo -S sysctl kernel.yama.ptrace_scope

kernel.yama.ptrace_scope = 0


[sudo] password for kali: 

In [8]:
# set the ptrace value
! echo kali | sudo -S sysctl -w kernel.yama.ptrace_scope=0

kernel.yama.ptrace_scope = 0


[sudo] password for kali: 

### 2. PIE
- Position Independent Executable
- randomizes Code segment base address
- randomizes GOT (Global Offset Table for global data-variables) and PLT (Procedure Linkage Table)
    - PLT addresses are fixed during  linking to load the libc.so library codes/functions
    - shared libraries are position independent code (PIC); they don't know where they are being loaded
- disable PIE in gcc/g++ using **-no-pie** flag

### 3. Executable Stack Protection (NX) also called (DEP)
- also called DEP - Data Execution Prevention
- in newer g++, programs must be compiled by disabling stack protector to execute code injected into stack
- check executable stack:
`$ readelf -l <filename>`
- note the line: 
GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10
RW 
- RW - Read and Write but not Executable
- RWE - Read, Write, and Executable

In [10]:
%%bash
in=./demos/hello.cpp
out=hello
g++ -g -m32 -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10


#### enable RWX/disable DEP
- compile the program using -z execstack switch of gcc/g++

In [11]:
%%bash
in=./demos/hello.cpp
out=hello
g++ -g -m32 -z execstack -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RWE 0x10


### 4. Stack Canaries
- "canary in a coal mine" to detect carbon monoxide by using a warm blooded animal like a bird
- place a small random integer just before the stack return address
    - if the value is overwritten/corrupted during the function call, there's a stack overflow overflow attempt!
- use **-fno-stack-protector** flag in gcc/g++ to disable canaries